Resolución de los incisos 1 y 5 del primer reporte grupal en Julia 1.7.2: 

In [1]:
# Installing relevant codes

using Pkg


Pkg.add("DataFrames")
Pkg.add("Dates")
Pkg.add("Plots")
Pkg.add("CategoricalArrays")
Pkg.add("Lathe")
Pkg.add("GLM") # package to run models 
Pkg.add("StatsPlots")
Pkg.add("MLBase")
Pkg.add("Tables")
Pkg.add("CovarianceMatrices")
Pkg.add("CSV")
Pkg.add("RData")



    Updating registry at `C:\Users\a2018\.julia\registries\General.toml`
   Resolving package versions...
   Installed Crayons ───────────────────── v4.1.1
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed PooledArrays ──────────────── v1.4.1
   Installed TableTraits ───────────────── v1.0.1
   Installed DataAPI ───────────────────── v1.9.0
   Installed DataValueInterfaces ───────── v1.0.0
   Installed Tables ────────────────────── v1.7.0
   Installed Formatting ────────────────── v0.4.2
   Installed OrderedCollections ────────── v1.4.1
   Installed InvertedIndices ───────────── v1.1.0
   Installed SortingAlgorithms ─────────── v1.0.1
   Installed Reexport ──────────────────── v1.2.2
   Installed Missings ──────────────────── v1.0.2
   Installed DataStructures ────────────── v0.18.11
   Installed Compat ────────────────────── v3.42.0
   Installed DataFrames ────────────────── v1.3.2
   Installed PrettyTables ──────────────── v1.3.1
    Updating `C:\Users\a2018\.julia\envir

LoadError: ArgumentError: Package RData not found in current path:
- Run `import Pkg; Pkg.add("RData")` to install the RData package.


In [1]:
# Calling relevant codes

using DataFrames
using Dates
using Plots
using Statistics,RData  #upload data of R format 
using CategoricalArrays # categorical data 
using CSV
using Tables
using Lathe
using GLM
using CovarianceMatrices


(process:5720): GLib-GIO-WARNING **: 13:31:56.447: Unexpectedly, UWP app `3138AweZip.AweZip_1.4.9.0_x86__ffd303wmbhcjt' (AUMId `3138AweZip.AweZip_ffd303wmbhcjt!App') supports 168 extensions but has no verbs

(process:5720): GLib-GIO-WARNING **: 13:31:56.559: Unexpectedly, UWP app `6760NGPDFLab.PDFX_1.3.15.0_x64__sbe4t8mqwq93a' (AUMId `6760NGPDFLab.PDFX_sbe4t8mqwq93a!App') supports 32 extensions but has no verbs


### Uploading database

In [2]:
rdata_read = load("../../data/wage2015_subsample_inference.RData")
data = rdata_read["data"]
names(data)
println("Number of Rows : ", size(data)[1],"\n","Number of Columns : ", size(data)[2],) #rows and columns

Number of Rows : 5150
Number of Columns : 20


## No control model

In [3]:
nocontrol_model = lm(@formula(lwage ~ sex), data)


StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

lwage ~ 1 + sex

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error       t  Pr(>|t|)   Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)   2.98783     0.0106588  280.32    <1e-99   2.96693     3.00873
sex          -0.0383447   0.0159878   -2.40    0.0165  -0.0696877  -0.0070018
─────────────────────────────────────────────────────────────────────────────

## OLS with controls 

In [4]:
flex = @formula(lwage ~ sex + (exp1+exp2+exp3+exp4) * (shs+hsg+scl+clg+occ2+ind2+mw+so+we))
control_model = lm(flex , data)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

lwage ~ 1 + sex + exp1 + exp2 + exp3 + exp4 + shs + hsg + scl + clg + occ2 + ind2 + mw + so + we + exp1 & shs + exp1 & hsg + exp1 & scl + exp1 & clg + exp1 & occ2 + exp1 & ind2 + exp1 & mw + exp1 & so + exp1 & we + exp2 & shs + exp2 & hsg + exp2 & scl + exp2 & clg + exp2 & occ2 + exp2 & ind2 + exp2 & mw + exp2 & so + exp2 & we + exp3 & shs + exp3 & hsg + exp3 & scl + exp3 & clg + exp3 & occ2 + exp3 & ind2 + exp3 & mw + exp3 & so + exp3 & we + exp4 & shs + exp4 & hsg + exp4 & scl + exp4 & clg + exp4 & occ2 + exp4 & ind2 + exp4 & mw + exp4 & so + exp4 & we

Coefficients:
────────────────────────────────────────────────────────────────────────────────────
                        Coef.  Std. Error      t  Pr(>|t|)    Lower 95%    Upper 95%
────────────────────────────────────────────────────────────────────────────

## The Partialling-Out model

In [5]:
# models
# model for Y
flex_y = @formula(lwage ~ (exp1+exp2+exp3+exp4) * (shs+hsg+scl+clg+occ2+ind2+mw+so+we))
flex_d = @formula(sex ~ (exp1+exp2+exp3+exp4) * (shs+hsg+scl+clg+occ2+ind2+mw+so+we))

# partialling-out the linear effect of W from Y
t_Y = residuals(lm(flex_y, data))

# partialling-out the linear effect of W from D
t_D = residuals(lm(flex_d, data))

data_res = DataFrame(t_Y = t_Y, t_D = t_D )
# regression of Y on D after partialling-out the effect of W

partial_fit = lm(@formula(t_Y ~ t_D), data_res)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

t_Y ~ 1 + t_D

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                    Coef.  Std. Error      t  Pr(>|t|)   Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)   1.16578e-14  0.00640313   0.00    1.0000  -0.0125528   0.0125528
t_D          -0.0695532    0.014853    -4.68    <1e-05  -0.0986714  -0.040435
──────────────────────────────────────────────────────────────────────────────

## Graphic of the interval confidence no control model

In [7]:
function coefplot(nocontrol_model)
       n = coefnames(nocontrol_model)[2:end] # no intercept
       vals = coef(nocontrol_model)[2:end]
       errors = stderr(nocontrol_model)[2:end]
       scatter(
           n,
           vals,
           legend = false,
           yerror = 1.96 .* errors,
           title = "Coefficient plot"
       )
       end

coefplot (generic function with 1 method)

In [8]:
coefplot(nocontrol_model)

LoadError: MethodError: objects of type IJulia.IJuliaStdio{Base.PipeEndpoint} are not callable

## Graphic of the interval confidence OLS - Controled model

In [9]:
function coefplot(control_model)
       n = coefnames(control_model)[2:end] # no intercept
       vals = coef(control_model)[2:end]
       errors = stderr(control_model)[2:end]
       scatter(
           n,
           vals,
           legend = false,
           yerror = 1.96 .* errors,
           title = "Coefficient plot"
       )
       end

coefplot (generic function with 1 method)

In [10]:
coefplot(control_model)

LoadError: MethodError: objects of type IJulia.IJuliaStdio{Base.PipeEndpoint} are not callable

## Graphic of the interval confidence OLS - Controled model

In [11]:
function coefplot(partial_fit)
       n = coefnames(partial_fit)[2:end] # no intercept
       vals = coef(partial_fit)[2:end]
       errors = stderr(partial_fit)[2:end]
       scatter(
           n,
           vals,
           legend = false,
           yerror = 1.96 .* errors,
           title = "Coefficient plot"
       )
       end

coefplot (generic function with 1 method)

In [12]:
coefplot(partial_fit)

LoadError: MethodError: objects of type IJulia.IJuliaStdio{Base.PipeEndpoint} are not callable